In [5]:
import numpy as np
import tensorflow as tf
import keras as k
from keras import Sequential
import random
from keras.layers import Embedding
from tensorflow.contrib.tensorboard.plugins import projector
%matplotlib inline
import matplotlib.pyplot as plt
import os
import math
import h5py
import pickle
import pdb
from progressbar import log_progress

log_dir = '/home/steich/NF_Prize_Data/log'

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
x_train = []
y_train = []
with open('../data/user_embedding_5_train.tsv', 'r') as iFile:
    for line in iFile:
        line = line.replace('\n', '')
        line = line.split('\t')
        x_train.append(eval(line[1]))
        y_train.append(int(line[0]))
        
epochs = 100
batch_size = 32
embedding_size = 64
num_sampled = 64

with open('../data/name_one_hot_dict.pickle', 'rb') as handle:
    name_one_hot = pickle.load(handle)
    
name_count = len(name_one_hot)

def generate_batch(train_features, train_labels, batch_size, steps):
    
    batch_samples = np.zeros((batch_size))
    batch_labels = np.zeros((batch_size, 1))
    index_list = list(range(len(train_labels)))
    random.shuffle(index_list)
    batch_count = 0
    step = 0
    
    while step < steps:
        
        curr_index = index_list[-1]
        for i in train_features[curr_index]:
            
            if batch_count == 32:
                batch_count = 0
                step += 1
                yield (batch_samples, batch_labels)
            
            batch_labels[batch_count][0] = name_one_hot[train_labels[curr_index]]
            batch_samples[batch_count] = name_one_hot[i]
            batch_count += 1
            
        index_list.pop()
        
graph = tf.Graph()
with graph.as_default():
    
    #input data here
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        #valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
        
    with tf.device('/gpu:0'):
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                tf.random_uniform([name_count, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
            
        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                tf.truncated_normal(
                    [name_count, embedding_size],
                    stddev=1.0 / math.sqrt(embedding_size)))
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([name_count]))
            
    
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                inputs=embed,
                labels=train_labels,
                num_sampled=num_sampled,
                num_classes=name_count))
        
    tf.summary.scalar('loss', loss)
    
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    
    #valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    #similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    merged = tf.summary.merge_all()
    
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    
with tf.Session(graph=graph) as session:
    
    writer = tf.summary.FileWriter(log_dir, session.graph)
    
    init.run()
    print('Initialized')
    
    steps = 1000
    average_loss = 0
    step = 0
    run_metadata = tf.RunMetadata()
    
    for epoch in range(epochs):
        if epoch > 0:
            print('Epoch: ', epoch, ' - Average Loss: ', average_loss / steps)
            average_loss = 0
        else:
            print('Epoch: ', epoch)
            
        for batch_inputs, batch_labels in log_progress(generate_batch(x_train, y_train, batch_size, steps),
                                                      every=1,
                                                      size=steps,
                                                      name="Batch"):
            
            feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

            run_metadata = tf.RunMetadata()

            _, summary, loss_val = session.run(
                [optimizer, merged, loss],
                feed_dict=feed_dict,
                run_metadata=run_metadata)
            average_loss += loss_val

            writer.add_summary(summary, step)
            step += 1
        
    writer.add_run_metadata(run_metadata, 'step%d' % step)
    final_embeddings = normalized_embeddings.eval()
    saver.save(session, os.path.join(log_dir, 'metadata.ckpt'))
    
writer.close()

from sklearn.manifold import TSNE
    
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
n = len(final_embeddings)
sample = random.sample(range(n), 500)
low_dim_embs = tsne.fit_transform([final_embeddings[i] for i in sample])
labels = sample
plt.figure(figsize=(18,18))
for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x,y)
    plt.annotate(label,
                    xy=(x, y),
                    xytext=(5,2),
                    textcoords='offset points',
                    ha='right',
                    va='bottom')
plt.savefig("/home/steich/NF_Prize_Data/data/tsne_user_embeddings_bow_all.png")
plt.show()
    
with open('/home/steich/NF_Prize_Data/data/user_emb_bow_all_unweighted.tsv', 'w') as f:
    for i, embs in enumerate(final_embeddings):
        emb_str = [str(emb) for emb in embs]
        f.write(str(i) + '\t' + '\t'.join(emb_str) + '\n')

In [ ]:
import numpy as np
import tensorflow as tf
import keras as k
from keras import Sequential
import random
from keras.layers import Embedding
from tensorflow.contrib.tensorboard.plugins import projector
%matplotlib inline
import matplotlib.pyplot as plt
import os
import math
import h5py
import pickle
import pdb
from progressbar import log_progress

log_dir = '/home/steich/NF_Prize_Data/log'

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
x_train = []
y_train = []
with open('../data/user_embedding_5_thresh_train.tsv', 'r') as iFile:
    for line in iFile:
        line = line.replace('\n', '')
        line = line.split('\t')
        x_train.append(eval(line[1]))
        y_train.append(int(line[0]))
        
epochs = 100
batch_size = 32
embedding_size = 64
num_sampled = 64

with open('../data/name_one_hot_dict.pickle', 'rb') as handle:
    name_one_hot = pickle.load(handle)
    
name_count = len(name_one_hot)

def generate_batch(train_features, train_labels, batch_size, steps):
    
    batch_samples = np.zeros((batch_size))
    batch_labels = np.zeros((batch_size, 1))
    index_list = list(range(len(train_labels)))
    random.shuffle(index_list)
    batch_count = 0
    step = 0
    
    while step < steps:
        
        curr_index = index_list[-1]
        for i in train_features[curr_index]:
            
            if batch_count == 32:
                batch_count = 0
                step += 1
                yield (batch_samples, batch_labels)
            
            batch_labels[batch_count][0] = name_one_hot[train_labels[curr_index]]
            batch_samples[batch_count] = name_one_hot[i]
            batch_count += 1
            
        index_list.pop()
        
graph = tf.Graph()
with graph.as_default():
    
    #input data here
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        #valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
        
    with tf.device('/gpu:0'):
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                tf.random_uniform([name_count, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
            
        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                tf.truncated_normal(
                    [name_count, embedding_size],
                    stddev=1.0 / math.sqrt(embedding_size)))
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([name_count]))
            
    
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                inputs=embed,
                labels=train_labels,
                num_sampled=num_sampled,
                num_classes=name_count))
        
    tf.summary.scalar('loss', loss)
    
    with tf.name_scope('optimizer'):
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    
    #valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    #similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    merged = tf.summary.merge_all()
    
    init = tf.global_variables_initializer()
    
    saver = tf.train.Saver()
    
with tf.Session(graph=graph) as session:
    
    writer = tf.summary.FileWriter(log_dir, session.graph)
    
    init.run()
    print('Initialized')
    
    steps = 1000
    average_loss = 0
    step = 0
    run_metadata = tf.RunMetadata()
    
    for epoch in range(epochs):
        if epoch > 0:
            print('Epoch: ', epoch, ' - Average Loss: ', average_loss / steps)
            average_loss = 0
        else:
            print('Epoch: ', epoch)
            
        for batch_inputs, batch_labels in log_progress(generate_batch(x_train, y_train, batch_size, steps),
                                                      every=1,
                                                      size=steps,
                                                      name="Batch"):
            
            feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

            run_metadata = tf.RunMetadata()

            _, summary, loss_val = session.run(
                [optimizer, merged, loss],
                feed_dict=feed_dict,
                run_metadata=run_metadata)
            average_loss += loss_val

            writer.add_summary(summary, step)
            step += 1
        
    writer.add_run_metadata(run_metadata, 'step%d' % step)
    final_embeddings = normalized_embeddings.eval()
    saver.save(session, os.path.join(log_dir, 'metadata.ckpt'))
    
writer.close()

from sklearn.manifold import TSNE
    
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
n = len(final_embeddings)
sample = random.sample(range(n), 500)
low_dim_embs = tsne.fit_transform([final_embeddings[i] for i in sample])
labels = sample
plt.figure(figsize=(18,18))
for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x,y)
    plt.annotate(label,
                    xy=(x, y),
                    xytext=(5,2),
                    textcoords='offset points',
                    ha='right',
                    va='bottom')
plt.savefig("/home/steich/NF_Prize_Data/data/tsne_user_embeddings_bow_thresh.png")
plt.show()
    
with open('/home/steich/NF_Prize_Data/data/user_emb_bow_thresh_unweighted.tsv', 'w') as f:
    for i, embs in enumerate(final_embeddings):
        emb_str = [str(emb) for emb in embs]
        f.write(str(i) + '\t' + '\t'.join(emb_str) + '\n')

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Initialized
Epoch:  0


VBox(children=(HTML(value=''), IntProgress(value=0, max=1000)))

Epoch:  1  - Average Loss:  282.0476734619141


VBox(children=(HTML(value=''), IntProgress(value=0, max=1000)))

...

VBox(children=(HTML(value=''), IntProgress(value=0, max=1000)))

Epoch:  86  - Average Loss:  40.39842712259293


VBox(children=(HTML(value=''), IntProgress(value=0, max=1000)))

Epoch:  87  - Average Loss:  40.20612968921662


VBox(children=(HTML(value=''), IntProgress(value=0, max=1000)))